# Апробация моделей частотных сканов на данных из набора "1564ЛЕ1 при разных U1 Ur август 2022"

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

## 0. Импорт модулей

In [ ]:
from os import listdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from pydlts.fsmodels import SklMultiExpFrequencyScan, SklSingleExpFrequencyScan

from pydlts.fsplots import plot_experimental_points
from pydlts.fsplots import plot_spectr
from pydlts.fsplots import plot_model
from pydlts.fsplots import plot_loss_path
from pydlts.fsplots import plot_deviations

%matplotlib inline

## 1. Чтение данных

In [ ]:
DATASETS_PATH = '../datasets/'
PLOTS_PATH = '../plots/'
MODELS_PATH = '../models/'

In [ ]:
file_names = listdir(DATASETS_PATH)

### 1.1. Выберем данные полученные при $U_1 = -4$ В и $U_R = -5$ В
Данному критерию соответствуют 3 частотных скана

In [ ]:
u1 = -4
ur = -5

data_frames = [[file_name, pd.read_csv(DATASETS_PATH + file_name, parse_dates=[0])] for file_name in file_names]

condition = lambda frame: frame.loc[0, 'u1']==u1 and frame.loc[0, 'ur']==ur
data_frames = [[name, frame] for [name, frame] in data_frames if condition(frame)]

In [ ]:
len(data_frames)

Отсортируем частотные сканы по температуре в начале сканирования

In [ ]:
key_func = lambda x: x[1].temperature_k.iloc[0]
data_frames.sort(key=key_func)

for i, item in enumerate(data_frames):
    print(f'{i}.', item[0])

Ниже приведён фрагмент таблицы с данными для частотного скана при температуре $T = -10^\circ C$

In [ ]:
print(data_frames[0][0])
data_frames[0][1].head()

Частотный скан содержит 340 точек. Данные не содержат пропусков.

In [ ]:
data_frames[0][1].info()

Ниже приведён фрагмент таблицы с данными для частотного скана при температуре $T = +10^\circ C$

In [ ]:
print(data_frames[1][0])
data_frames[1][1].head()

Частотный скан содержит 340 точек. Данные не содержат пропусков.

In [ ]:
data_frames[1][1].info()

Ниже приведён фрагмент таблицы с данными для частотного скана при температуре $T = +30^\circ C$

In [ ]:
print(data_frames[2][0])
data_frames[2][1].head()

Частотный скан содержит 34 точки. Данные не содержат пропусков.

In [ ]:
data_frames[2][1].info()

## 2. 1564ЛЕ1№1\_п1\_2500Гц-1Гц\_10пФ\_-10С\_-4В-5В\_50мВ\_10мкс\_шаг\_0,1.csv
### 2.1. Подготовка экспериментальных данных

Чтение данных

In [ ]:
print(data_frames[0][0])

In [ ]:
def get_X_y(dataset):
    filling_pulse = dataset.f_pulse[0] * 10 ** -6
    
    df = dataset.sample(frac = 1, ignore_index=True)
    X_train = np.log10(df.frequency_hz.to_numpy())
    y_train = df.dlts_pf.to_numpy()
    
    return filling_pulse, X_train, y_train

In [ ]:
filling_pulse, X_train, y_train = get_X_y(data_frames[0][1])

Нормализация данных

In [ ]:
normalize = lambda X, y: (X.max() / y.max(), y * X.max() / y.max())
denormalize = lambda y_normalized, k: y_normalized / k

In [ ]:
k, y_train = normalize(X_train, y_train)
print(f'Нормализующий коэффицет: {k}')

In [ ]:
fig, ax = plot_experimental_points(X_train, denormalize(y_train, k))
ax.set_xlim([0, 3.5])
ax.set_ylabel('Сигнал DLTS, пФ')
plt.show()

### 2.2. Идентификация моноэкспоненциальной модели с показателем $p$
Идентификация модели:

In [ ]:
monoexp_fs = SklSingleExpFrequencyScan(filling_pulse = filling_pulse,
                                       learning_rate = 0.05,
                                       stop_val = 10**-5,
                                       verbose = False
                                      )

monoexp_fs.fit(X = X_train, y = y_train, initial_exps_params_=[[-3.0, 0.5]])

Приведение результатов к изначальному масштабу

In [ ]:
exps_params_ = monoexp_fs.exps_params_
exps_params_[0, 1] = denormalize(exps_params_[0, 1], k)
monoexp_fs.exps_params_ = exps_params_

fit_results_ = monoexp_fs.fit_results_.copy()
fit_results_.amplitude_0 = denormalize(fit_results_.amplitude_0, k) 

y_train = denormalize(y_train, k)

Вывод табличных результатов идентификации

In [ ]:
fit_results_.to_csv(MODELS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_fit_results_1.csv')
monoexp_fs.fit_results_.tail(10)

Вывод графика с экспериментальными данными и данными полученными на идентифицированной модели

In [ ]:
fig, ax = plot_model(X_train, y_train, SklSingleExpFrequencyScan, fit_results_, plot_exps=False)
ax.set_ylabel('Сигнал DLTS, пФ')
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_model_1.pdf',
            bbox_inches='tight')
plt.show()

Вывод графика значений среднеквадратической ошибки в процессе идентификации.

In [ ]:
plot_loss_path(fit_results_)
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_model_loss_1.pdf',
            bbox_inches='tight')
plt.show()

График выходит на "плато", что свидетельствует о том, что решение сошлось, однако довольно высокое значение среднеквадратической ошибки может указывать на то, что алгоритм нашёл локальный минимум функции потерь. О последнем также свидетельсвует предыдущий график (графика с экспериментальными данными и данными полученными на идентифицированной модели).

Выведем график отклонений результатов, полученных на модели, от экспериментальных данных.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax = plot_deviations(X_train, y_train, monoexp_fs.predict(X_train), ax=ax)

ax.set_ylabel('Сигнал DLTS, пФ')

plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_deviations_1.pdf',
            bbox_inches='tight')
plt.show()

Построим гистограмму отклонений данных, полученных на идентифицированной модели, от экспериментальных данных

In [ ]:
sns.displot(x=y_train - monoexp_fs.predict(X_train), bins=11, kde=True, stat='probability', aspect=1.5)
plt.grid()
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_hist_1.pdf',
            bbox_inches='tight')
plt.show()

Оценим модель при помощи кросс-валидации

In [ ]:
def display_scores(scores):
    print('Scores - корни из среднеквадратических ошибок')
    scores = np.sqrt(scores)
    print(f'Scores: {scores}')
    print(f'Mean: {scores.mean()}')
    print(f'Standard deviation: {scores.std()}')

In [ ]:
scores = cross_val_score(monoexp_fs, 
                         X_train, 
                         y_train, 
                         fit_params=dict(initial_exps_params_=[[-3.0, 0.5]]), 
                         scoring='neg_mean_squared_error', 
                         cv=5, 
                         n_jobs=-1)

display_scores(np.sqrt(-scores))

Оценим корень из среднеквадратической ошики между экспериментальными данными и данными, полученными на идентифицированной модели

In [ ]:
mean_squared_error(y_train, monoexp_fs.predict(X_train), squared=False)

Выведем на экран полученный спектр

In [ ]:
plot_spectr(monoexp_fs.exps_params_, xlim=[1/5000, 1])
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_spectr_1.pdf',
            bbox_inches='tight')
plt.show()

Попробуем идентифицировать модель задав другую начальную точку для идентификации.

Чтение данных

In [ ]:
filling_pulse, X_train, y_train = get_X_y(data_frames[0][1])

Нормализация данных

In [ ]:
k, y_train = normalize(X_train, y_train)
print(f'Нормализующий коэффицет: {k}')

Идентификация модели с начальной точкой близкой к отрицательному пику

In [ ]:
monoexp_fs = SklSingleExpFrequencyScan(filling_pulse = filling_pulse,
                                       learning_rate = 0.05,
                                       stop_val = 10**-5,
                                       verbose = False
                                      )

monoexp_fs.fit(X = X_train, y = y_train, initial_exps_params_=[[-1.5, -0.5]])

Приведение результатов к изначальному масштабу

In [ ]:
exps_params_ = monoexp_fs.exps_params_
exps_params_[0, 1] = denormalize(exps_params_[0, 1], k)
monoexp_fs.exps_params_ = exps_params_

fit_results_ = monoexp_fs.fit_results_.copy()
fit_results_.amplitude_0 = denormalize(fit_results_.amplitude_0, k) 

y_train = denormalize(y_train, k)

Вывод табличных результатов идентификации

In [ ]:
fit_results_.to_csv(MODELS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_fit_results_2.csv')
monoexp_fs.fit_results_.tail(10)

Вывод графика с экспериментальными данными и данными полученными на идентифицированной модели

In [ ]:
fig, ax = plot_model(X_train, y_train, SklSingleExpFrequencyScan, fit_results_, plot_exps=False)
ax.set_ylabel('Сигнал DLTS, пФ')
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_model_2.pdf',
            bbox_inches='tight')
plt.show()

Вывод графика значений среднеквадратической ошибки в процессе идентификации.

In [ ]:
plot_loss_path(fit_results_)
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_model_loss_2.pdf',
            bbox_inches='tight')
plt.show()

Выведем график отклонений результатов, полученных на модели, от экспериментальных данных.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax = plot_deviations(X_train, y_train, monoexp_fs.predict(X_train), ax=ax)

ax.set_ylabel('Сигнал DLTS, пФ')

plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_deviations_2.pdf',
            bbox_inches='tight')
plt.show()

Построим гистограмму отклонений данных, полученных на идентифицированной модели, от экспериментальных данных

In [ ]:
sns.displot(x=y_train - monoexp_fs.predict(X_train), bins=11, kde=True, stat='probability', aspect=1.5)
plt.grid()
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_hist_2.pdf',
            bbox_inches='tight')
plt.show()

Оценим модель при помощи кросс-валидации

In [ ]:
scores = cross_val_score(monoexp_fs, 
                         X_train, 
                         y_train, 
                         fit_params=dict(initial_exps_params_=[[-1.5, -0.5]]), 
                         scoring='neg_mean_squared_error', 
                         cv=5, 
                         n_jobs=-1)

display_scores(np.sqrt(-scores))

Оценим корень из среднеквадратической ошики между экспериментальными данными и данными, полученными на идентифицированной модели

In [ ]:
mean_squared_error(y_train, monoexp_fs.predict(X_train), squared=False)

Выведем на экран полученный спектр

In [ ]:
plot_spectr(monoexp_fs.exps_params_)
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_spectr_2.pdf',
            bbox_inches='tight')
plt.show()

### 2.3. Идентификация моноэкспоненциальной моделью с показателем $p=1$

Чтение данных

In [ ]:
filling_pulse, X_train, y_train = get_X_y(data_frames[0][1])

Нормализация данных

In [ ]:
k, y_train = normalize(X_train, y_train)
print(f'Нормализующий коэффицет: {k}')

Идентификация модели

In [ ]:
monoexp_fs = SklSingleExpFrequencyScan(filling_pulse = filling_pulse,
                                       learning_rate = 0.05,
                                       stop_val = 10**-5,
                                       verbose = False,
                                       fit_p_coef = False
                                      )

monoexp_fs.fit(X = X_train, y = y_train)

Приведение результатов к изначальному масштабу

In [ ]:
exps_params_ = monoexp_fs.exps_params_
exps_params_[0, 1] = denormalize(exps_params_[0, 1], k)
monoexp_fs.exps_params_ = exps_params_

fit_results_ = monoexp_fs.fit_results_.copy()
fit_results_.amplitude_0 = denormalize(fit_results_.amplitude_0, k) 

y_train = denormalize(y_train, k)

Вывод табличных результатов идентификации

In [ ]:
fit_results_.to_csv(MODELS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_ideal_fit_results.csv')
monoexp_fs.fit_results_.tail(10)

In [ ]:
fig, ax = plot_model(X_train, y_train, SklSingleExpFrequencyScan, fit_results_, plot_exps=False)
ax.set_ylabel('Сигнал DLTS, пФ')
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_ideal_model.pdf',
            bbox_inches='tight')
plt.show()

Вывод графика значений среднеквадратической ошибки в процессе идентификации.

In [ ]:
plot_loss_path(fit_results_)
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_ideal_model_loss.pdf',
            bbox_inches='tight')
plt.show()

Выведем график отклонений результатов, полученных на модели, от экспериментальных данных.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax = plot_deviations(X_train, y_train, monoexp_fs.predict(X_train), ax=ax)

ax.set_ylabel('Сигнал DLTS, пФ')

plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_ideal_deviations.pdf',
            bbox_inches='tight')
plt.show()

Построим гистограмму отклонений данных, полученных на идентифицированной модели, от экспериментальных данных

In [ ]:
sns.displot(x=y_train - monoexp_fs.predict(X_train), bins=11, kde=True, stat='probability', aspect=1.5)
plt.grid()
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_ideal_hist.pdf',
            bbox_inches='tight')
plt.show()

Оценим модель при помощи кросс-валидации

In [ ]:
scores = cross_val_score(monoexp_fs, 
                         X_train, 
                         y_train, 
                         scoring='neg_mean_squared_error', 
                         cv=5, 
                         n_jobs=-1)

display_scores(np.sqrt(-scores))

Оценим корень из среднеквадратической ошики между экспериментальными данными и данными, полученными на идентифицированной модели

In [ ]:
mean_squared_error(y_train, monoexp_fs.predict(X_train), squared=False)

Выведем на экран полученный спектр

In [ ]:
plot_spectr(monoexp_fs.exps_params_, xlim=[1/5000, 1])
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_single_exp_ideal_spectr.pdf',
            bbox_inches='tight')
plt.show()

### 2.4. Идентификация мультиэкспоненциальной модели с n_exps>1

Чтение данных

In [ ]:
print(data_frames[0][0])

In [ ]:
filling_pulse, X_train, y_train = get_X_y(data_frames[0][1])

Нормализация данных

In [ ]:
k, y_train = normalize(X_train, y_train)
print(f'Нормализующий коэффицет: {k}')

Идентификация мультиэкспоненциальной модели

In [ ]:
multiexp_fs = SklMultiExpFrequencyScan(n_exps = 1,
                                       filling_pulse = filling_pulse,
                                       learning_rate = 0.05,
                                       stop_val = 10**-5
                                      )

In [ ]:
param_grid = {'n_exps': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
grid_search = GridSearchCV(multiexp_fs, param_grid, cv=5, 
                           scoring='neg_mean_squared_error',
                           return_train_score=True, n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

Лучший результат показала модель со следующим количеством экспоненциальных составляющих (n_exps)

In [ ]:
grid_search.best_params_

Ниже приведены результаты, которые показали модули с другими занчениями параметра (n_exps)

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

Для лучшей модели выведем табличные результаты и графики.

Приведение результатов к изначальному масштабу

In [ ]:
multiexp_fs = grid_search.best_estimator_

exps_params_ = multiexp_fs.exps_params_
exps_params_[:, 1] = denormalize(exps_params_[:, 1], k)
multiexp_fs.exps_params_ = exps_params_

fit_results_ = multiexp_fs.fit_results_.copy()

for i in range(multiexp_fs.n_exps):
    fit_results_[f'amplitude_{i}'] = denormalize(fit_results_[f'amplitude_{i}'], k) 

y_train = denormalize(y_train, k)

Вывод табличных результатов идентификации

In [ ]:
fit_results_.to_csv(MODELS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_multi_exp_fit_results_1.csv')
multiexp_fs.fit_results_.tail(10)

Вывод графика с экспериментальными данными и данными полученными на идентифицированной модели

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_train)

In [ ]:
fig, ax = plot_model(X_train, y_train, SklMultiExpFrequencyScan, fit_results_)
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_multi_exp_model_2.pdf',
            bbox_inches='tight')
plt.show()

Вывод графика значений среднеквадратической ошибки в процессе идентификации

In [ ]:
plot_loss_path(fit_results_)
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_multi_exp_loss.pdf',
            bbox_inches='tight')
plt.show()

Выведем график отклонений результатов, полученных на модели, от экспериментальных данных

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax = plot_deviations(X_train, y_train, multiexp_fs.predict(X_train), ax=ax)

ax.set_ylabel('Сигнал DLTS, пФ')

plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_multi_exp_deviations.pdf',
            bbox_inches='tight')
plt.show()

Построим гистограмму отклонений данных, полученных на идентифицированной модели, от экспериментальных данных

In [ ]:
sns.displot(x=y_train - multiexp_fs.predict(X_train), bins=11, kde=True, stat='probability', aspect=1.5)
plt.grid()
plt.savefig(PLOTS_PATH + '1564ЛЕ1№1_п1_2500Гц-1Гц_10пФ_+30С_-4В-5В_50мВ_10мкс_шаг_0,1_multi_exp_hist.pdf',
            bbox_inches='tight')
plt.show()

Оценим модель при помощи кросс-валидации

In [ ]:
scores = cross_val_score(monoexp_fs, 
                         X_train, 
                         y_train, 
                         scoring='neg_mean_squared_error', 
                         cv=5, 
                         n_jobs=-1)

display_scores(np.sqrt(-scores))

Оценим корень из среднеквадратической ошики между экспериментальными данными и данными, полученными на идентифицированной модели

In [ ]:
mean_squared_error(y_train, multiexp_fs.predict(X_train), squared=False)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
# https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

## 3. 1564ЛЕ1№1\_п1\_2500Гц-1Гц\_1пФ\_+10С\_-4В-5В\_50мВ\_10мкс\_шаг\_0,01.csv
### 3.1. Подготовка экспериментальных данных

In [ ]:
filling_pulse, X_train, y_train = get_X_y(data_frames[1][1])

In [ ]:
k, y_train = normalize(X_train, y_train)
k

In [ ]:
fig, ax = plot_experimental_points(X_train, y_train)
ax.set_ylim([0, 3.5])
ax.set_xlim([0, 3.5])
plt.show()

### 3.2. Идентификация моноэкспоненциальной модели с показателем $p$

In [ ]:
monoexp_fs = SklSingleExpFrequencyScan(filling_pulse = filling_pulse,
                                       learning_rate = 0.05,
                                       stop_val = 10**-5,
                                       verbose = False
                                      )

monoexp_fs.fit(X = X_train, y = y_train)
monoexp_fs.fit_results_

In [ ]:
fig, ax = plot_model(X_train, y_train, SklSingleExpFrequencyScan, monoexp_fs.fit_results_, plot_exps=False)
plt.show()

In [ ]:
plot_deviations(X_train, y_train, monoexp_fs.predict(X_train))
plt.show()

In [ ]:
sns.displot(x=y_train - monoexp_fs.predict(X_train), bins=7, kde=True)
plt.show()

In [ ]:
sns.displot(x=y_train - monoexp_fs.predict(X_train), bins=5, kde=True)
plt.grid()
plt.show()

In [ ]:
mean_squared_error(y_train, y_train - monoexp_fs.predict(X_train), squared=False)

### 3.3. Идентификация мультиэкспоненциальной модели с n_exps=1

In [ ]:
multiexp_fs = SklMultiExpFrequencyScan(n_exps = 1,
                                       filling_pulse = filling_pulse,
                                       stop_val = 10**-5,
                                       learning_rate=0.05,
                                      )

multiexp_fs.fit(X=X_train, y=y_train)

In [ ]:
multiexp_fs.fit_results_

In [ ]:
fig, ax = plot_model(X_train, y_train, SklMultiExpFrequencyScan, multiexp_fs.fit_results_, plot_exps=False)
plt.show()

In [ ]:
plot_deviations(X_train, y_train, multiexp_fs.predict(X_train))
plt.show()

In [ ]:
sns.displot(x=y_train - multiexp_fs.predict(X_train), bins=7, kde=True)
plt.grid()
plt.show()

In [ ]:
sns.displot(x=y_train - multiexp_fs.predict(X_train), bins=5, kde=True)
plt.show()

In [ ]:
scores = cross_val_score(multiexp_fs, X_train, y_train, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

display_scores(np.sqrt(-scores))

In [ ]:
mean_squared_error(y_train, multiexp_fs.predict(X_train), squared=False)

### 3.4. Идентификация мультиэкспоненциальной модели с n_exps>1

In [ ]:
multiexp_fs = SklMultiExpFrequencyScan(n_exps = 1,
                                       filling_pulse = filling_pulse,
                                       learning_rate = 0.05,
                                       stop_val = 10**-5
                                      )

In [ ]:
param_grid = {'n_exps': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
grid_search = GridSearchCV(multiexp_fs, param_grid, cv=5, 
                           scoring='neg_mean_squared_error',
                           return_train_score=True, n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_train)

In [ ]:
fig, ax = plot_model(X_train, y_train, SklMultiExpFrequencyScan, grid_search.best_estimator_.fit_results_)

plt.show()

In [ ]:
grid_search.best_estimator_.fit_results_

In [ ]:
plot_deviations(X_train, y_train, y_pred)
plt.show()

In [ ]:
sns.displot(x=y_train - y_pred, bins=7, kde=True)
plt.show()

In [ ]:
sns.displot(x=y_train - y_pred, bins=5, kde=True)
plt.show()

In [ ]:
exps_params_283 = grid_search.best_estimator_.exps_params_
exps_params_283[:, 1] = exps_params_283[:, 1] / k

In [ ]:
mean_squared_error(y_train, y_pred, squared=False)

## 4. 1564ЛЕ1№1\_п1\_2500Гц-1Гц\_1пФ\_-10С\_-4В-5В\_10мВ\_10мкс\_шаг\_0,01.csv
### 4.1. Подготовка экспериментальных данных

In [ ]:
filling_pulse, X_train, y_train = get_X_y(data_frames[2][1])

In [ ]:
k, y_train = normalize(X_train, y_train)
k

In [ ]:
fig, ax = plot_experimental_points(X_train, y_train)
plt.show()

### 4.2. Идентификация моноэкспоненциальной модели с показателем $p$

In [ ]:
monoexp_fs = SklSingleExpFrequencyScan(filling_pulse = filling_pulse,
                                       learning_rate = 0.05,
                                       stop_val = 10**-5,
                                       verbose = False
                                      )

monoexp_fs.fit(X = X_train, y = y_train)
monoexp_fs.fit_results_

In [ ]:
fig, ax = plot_model(X_train, y_train, SklSingleExpFrequencyScan, monoexp_fs.fit_results_, plot_exps=False)
plt.show()

In [ ]:
plot_deviations(X_train, y_train, monoexp_fs.predict(X_train))
plt.show()

In [ ]:
sns.displot(x=y_train - monoexp_fs.predict(X_train), bins=7, kde=True)
plt.show()

In [ ]:
mean_squared_error(y_train, y_train - monoexp_fs.predict(X_train), squared=False)

### 4.3. Идентификация мультиэкспоненциальной модели с n_exps=1

In [ ]:
multiexp_fs = SklMultiExpFrequencyScan(n_exps = 1,
                                       filling_pulse = filling_pulse,
                                       stop_val = 10**-5,
                                       learning_rate=0.05,
                                      )

multiexp_fs.fit(X=X_train, y=y_train)

In [ ]:
multiexp_fs.fit_results_

In [ ]:
fig, ax = plot_model(X_train, y_train, SklMultiExpFrequencyScan, multiexp_fs.fit_results_, plot_exps=False)
plt.show()

In [ ]:
plot_deviations(X_train, y_train, multiexp_fs.predict(X_train))
plt.show()

In [ ]:
sns.displot(x=y_train - multiexp_fs.predict(X_train), bins=7, kde=True)
plt.show()

In [ ]:
scores = cross_val_score(multiexp_fs, X_train, y_train, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
display_scores(np.sqrt(-scores))

In [ ]:
mean_squared_error(y_train, multiexp_fs.predict(X_train), squared=False)

### 4.4. Идентификация мультиэкспоненциальной модели с n_exps>1

In [ ]:
multiexp_fs = SklMultiExpFrequencyScan(n_exps = 1,
                                       filling_pulse = filling_pulse,
                                       learning_rate = 0.05,
                                       stop_val = 10**-5
                                      )

In [ ]:
param_grid = {'n_exps': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
grid_search = GridSearchCV(multiexp_fs, param_grid, cv=5, 
                           scoring='neg_mean_squared_error',
                           return_train_score=True, n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_train)

In [ ]:
fig, ax = plot_model(X_train, y_train, SklMultiExpFrequencyScan, grid_search.best_estimator_.fit_results_)
plt.show()

In [ ]:
grid_search.best_estimator_.fit_results_

In [ ]:
plot_deviations(X_train, y_train, y_pred)
plt.show()

In [ ]:
sns.displot(x=y_train - y_pred, bins=7, kde=True)
plt.show()

In [ ]:
sns.displot(x=y_train - y_pred, bins=21, kde=True)
plt.show()

In [ ]:
exps_params_263 = grid_search.best_estimator_.exps_params_
exps_params_263[:, 1] = exps_params_303[:, 1] / k